# Chapter 10
. Sequence Modeling: Recurrentand Recursive Nets

* 손고리즘ML : 파트 4 - DML [1]
* 김무성

* 10.1 Unfolding Computational Graphs
* 10.2 Recurrent Neural Networks
    - 10.2.1 Computing the Gradient in a Recurrent Neural Network
    - 10.2.2 Recurrent Networks as Directed Graphical Models
    - 10.2.3 Modeling Sequences Conditioned on Context with RNNs
* 10.3 Bidirectional RNNs
* 10.4 Encoder-Decoder Sequence-to-Sequence Architectures
* 10.5 Deep Recurrent Networks
* 10.6 Recursive Neural Networks
* 10.7 The Challenge of Long-Term Dependencies
* 10.8 Echo State Networks
* 10.9 Skip Connections through Time
* 10.10 Leaky Units and a Spectrum of Diﬀerent Time Scales
* 10.11 The Long Short-Term Memory and Other Gated RNNs
    - 10.11.1 LSTM
    - 10.11.2 Other Gated RNNs
* 10.12 Optimization for Long-Term Dependencies
    - 10.12.1 Clipping Gradients
* 10.13 Regularizing to Encourage Information Flow
* 10.14 Organizing the State at Multiple Time Scales
* 10.15 Explicit Memory

#### Recurrent neural networks

#### 참고
* [2] CS224d: Deep Learning for Natural Language Processing : Recurrent neural networks -- for language modeling and other tasks - http://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf
* [3] CS231n: Convolutional Neural Networks for Visual Recognition (2015) - Beyond Image Classification: localization, detection, segmentation - http://vision.stanford.edu/teaching/cs231n/slides/lecture11.pdf 
Recurrent Networks I: Image Captioning example
* [4] CS231n: Convolutional Neural Networks for Visual Recognition (2016) - Recurrent Neural Networks (RNN), Long Short Term Memory (LSTM) - http://cs231n.stanford.edu/slides/winter1516_lecture10.pdf
* [5] Probabilistic Graphical Models : Template Models - http://spark-university.s3.amazonaws.com/stanford-pgm/slides/Section-2-Representation-Template-Models.pdf
* [13] Recurrent Neural Network - https://piazza.com/class_profile/get_resource/i48o74a0lqu0/i6ys94c8na8i2

Recurrent neural networks or RNNs (Rumelhart et al., 1986a) are a family of neural networks for handling sequential data. Sequential data is data where each example consists of a sequence, with each example able to have a different sequence length.

#### computational graph

This chapter extends the idea of a computational graph, introduced in Sec. 6.5.1, to include cycles. These cycles represent the influence of the present value of a variable on its own value at a future time step. Such computational graphs allow us to define recurrent neural networks. We then describe many different ways to construct, train, and use recurrent neural networks.

# 10.1 Unfolding Computational Graphs

* In this section we explain the idea of unfolding a recursive or recurrent computation into a computational graph that has a repetitive structure, typically corresponding to a chain of events
* Unfolding this graph results in the sharing of parameters across a deep network structure.

#### classical form of dynamical system

For example, consider the classical form of a dynamical system:

<img src="figures/cap10.1.png" width=600 />

where s(t) is called the state of the system.

#### unfolding

For a finite number of time steps τ, the graph can be unfolded by applying the
definition τ − 1 times. For example, if we unfold Eq. 10.1 for τ = 3 time steps, we obtain

<img src="figures/cap10.2.png" width=600 />

The unfolded computational graph of Eq. 10.1 and Eq. 10.2 is illustrated in Fig. 10.1.

<img src="figures/cap10.3.png" width=600 />

#### external signal

As another example, let us consider a dynamical system driven by an external
signal x(t),

<img src="figures/cap10.4.png" width=600 />

where we see that the state now contains information about the whole past sequence.

#### hidden state

To indicate that the state is the hidden units of
the network, we now rewrite Eq. 10.3 using the variable h to represent the state:

<img src="figures/cap10.5.png" width=600 />

illustrated in Fig. 10.2, Typical RNNs will add extra architectural features such as
output layers that read information out of the state h to make predictions.

#### circuit diagram (recurrent graph) or unfolded computational graph (unrolled graph)

Eq. 10.4 can be drawn in two different ways. 
* One way to draw the RNN is with a diagram containing one node for every component that might exist in a physical implementation of the model, such as a biological neural network.
* The other way to draw the RNN is as an unfolded computational graph, in which each component is represented by many different variables, with one variable per time step, representing the state of the component at that point in time.

<img src="figures/cap10.6.png" width=600 />

The unfolding process offers two major advantages over simply constructing a
function of the full sequence x(t) for t = 1,...,τ.

We can represent the unfolded recurrence after t steps with a function g(t):

<img src="figures/cap10.7.png" width=600 />

The function g(t) takes the whole past sequence (x(t), x(t−1) , x(t−2), . . . , x(2), x(1) ) as input and produces the current state.

Another advantage of the unfolding process is that the same function f with <font color="red">the same parameters θ</font> is used at each time step.

#### Both the recurrent graph and the unrolled graph have their uses. 
* The recurrent graph is succint. 
* The unfolded graph provides an explicit description of which computations to perform. 
    - The unfolded graph also helps to illustrate the idea of information flow forward in time (computing outputs and losses) and backward in time (computing gradients) by explicitly showing the path along which this information flows.

# 10.2 Recurrent Neural Networks
   * 10.2.1 Computing the Gradient in a Recurrent Neural Network
   * 10.2.2 Recurrent Networks as Directed Graphical Models
   * 10.2.3 Modeling Sequences Conditioned on Context with RNNs

Some examples of important design patterns for recurrent neural networks include the following:
* Recurrent networks that <font color="red">produce an output at each time step</font> and have <font color="blue">recurrent connections between hidden units</font>, illustrated in Fig. 10.3.

<img src="figures/cap10.8.png" width=600 />

* Recurrent networks that produce an output at each time step and have <font color="red">recurrent connections only from the output at one time step to the hidden units at the next time step</font>, illustrated in Fig. 10.4

<img src="figures/cap10.10.png" width=600 />

* Recurrent networks with recurrent connections between hidden units, that <font color="red">read an entire sequence and then produce a single output</font>, illustrated in Fig. 10.5.

<img src="figures/cap10.12.png" width=600 />

#### forward propagation equations

* We now develop the forward propagation equations for the RNN depicted in Fig. 10.3.
* Here we assume the hyperbolic tangent activation function.
* Here we assume that the output is discrete, as if the RNN is used to predict words or characters.
* A natural way to represent discrete variables is to regard the output o as giving the unnormalized log probabilities of each possible value of the discrete variable.
* We can then apply the softmax operation as a post-processing step to obtain a vector ˆy of normalized probabilities over the output.
* Forward propagation begins with a specification of the initial state h(0).

<img src="figures/cap10.8.png" width=600 />

Then, for each time step from t = 1 to t = τ, we apply the following update equations:

<img src="figures/cap10.9.png" width=600 />

where the parameters are the bias vectors b and c along with the weight matrices U, V and W, respectively for input-to-hidden, hidden-to-output and hidden-to- hidden connections.

#### total loss

* This is an example of a recurrent network that maps an input sequence to an output sequence of the same length. 
* The total loss for a given sequence of x values paired with a sequence of y values would then be just the sum of the losses over all the time steps.

For example, if L(t) is the negative log-likelihood of y(t) given x(1),...,x(t),then

<img src="figures/cap10.11.png" width=600 />

* Computing the gradient of this loss function with respect to the parameters is an expensive operation.
* The runtime is O (τ) and cannot be reduced by parallelization because the forward propagation graph is inherently sequential; each time step may only be computed after the previous one.
* States computed in the forward pass must be stored until they are reused during the backward pass, so the memory cost is also O(τ).

#### back-propagation through time (BPTT)

* The back-propagation algorithm applied to the unrolled graph with O(τ) cost is called back-propagation through time or BPTT and is discussed further Sec.
10.2.1.

<font color="red">The network with recurrence between hidden units is thus very powerful but also expensive to train. Is there an alternative?</font>

The network with recurrent connections only from the output at one time step to the hidden units at the next time step (shown in Fig. 10.4) is strictly <font color="red">less powerful</font> because it lacks hidden-to-hidden recurrent connections.
* Because this network lacks hidden-to- hidden recurrence, it requires that the output units capture all of the information about the past that the network will use to predict the future.

<font color="red">The advantage of eliminating hidden-to-hidden recurrence</font> is is that, for any loss function based on comparing the prediction at time t to the training target at time t , all the time steps are decoupled. Training can thus be parallelized, with the gradient for each step t computed in isolation.

<img src="figures/cap10.10.png" width=600 />

#### teacher forcing

* Models that have recurrent connections from their outputs leading back into the model may be trained with teacher forcing.
* Teacher forcing is a procedure that emerges from the maximum likelihood criterion, in which during training the model receives the ground truth output y(t) as input at time t+ 1.

We can see this by examining a sequence with two time steps.

The conditional maximum likelihood criterion is

<img src="figures/cap10.13.png" width=600 />

<img src="figures/cap10.14.png" width=600 />

* We originally motivated teacher forcing as allowing us to avoid back-propagation through time in models that lack hidden-to-hidden connections. 
* Teacher forcing may still be applied to models that have hidden-to-hidden connections so long as they have connections from the output at one time step to values computed in the next time step. 
* However, as soon as the hidden units become a function of earlier time steps, the BPTT algorithm is necessary. Some models may thus be trained with both teacher forcing and BPTT.

The disadvantage of strict teacher forcing arises if the network is going to be later used in an open-loop mode, with the network outputs (or samples from the output distribution) fed back as input.
* In this case, the kind of inputs that the network sees during training could be quite different from the kind of inputs that it will see at test time.
* One way to mitigate this problem is to train with both teacher-forced inputs and with free-running inputs, for example by predicting the correct target a number of steps in the future through the unfolded recurrent output-to-input paths. 
    - In this way, the network can learn to take into account input conditions (such as those it generates itself in the free-running mode) not seen during training and how to map the state back towards one that will make the network generate proper outputs after a few steps. 
* Another approach (Bengio et al., 2015b) to mitigate the gap between the inputs seen at train time and the inputs seen at test time randomly chooses to use generated values or actual data values as input. 
    - This approach exploits a curriculum learning strategy to gradually use more of the generated values as input.

## 10.2.1 Computing the Gradient in a Recurrent Neural Network

Computing the gradient through a recurrent neural network is straightforward. One simply applies the generalized back-propagation algorithm of Sec. 6.5.6 to the unrolled computational graph.

The use of back-propagation on the unrolled graph is called <font color="red">the back-propagation through time (BPTT) algorithm</font>.

#### How to compute gradients by BPTT

To gain some intuition for how the BPTT algorith behaves, we provide an example of how to compute gradients by BPTT for the RNN equations above (Eqs. 10.6 and 10.7).

<img src="figures/cap10.9.png" width=600 />
<img src="figures/cap10.11.png" width=600 />

<img src="figures/cap10.8.png" width=600 />

For each node N we need to compute the gradient ∇NL recursively, based on the gradient computed at nodes that follow it in the graph.

We start the recursion with the nodes immediately preceding the final loss

<img src="figures/cap10.15.png" width=600 />

The gradient ∇o(t)L on the outputs at time step t, for all i,t, is as follows:

<img src="figures/cap10.16.png" width=600 />

At the final time step τ, h (τ) only has o(τ) as a descendent, so its gradient is simple:

<img src="figures/cap10.17.png" width=600 />

We can then iterate backwards in time to back-propagate gradients through time, from t = τ − 1 down to t = 1, noting that h(t) (for t < τ ) has as descendents both o(t) and h(t+1). Its gradient is thus given by

<img src="figures/cap10.18.png" width=600 />

where diag(1 − (h(t+1)^2) indicates the diagonal matrix containing the elements 1 − (h_i(t+1))^2 . This is the Jacobian of the hyperbolic tangent associated with the hidden unit i at time t + 1

Once the gradients on the internal nodes of the computational graph are obtained, we can obtain the gradients on the parameter nodes, which have descendents at all the time steps:

<img src="figures/cap10.19.png" width=600 />

## 10.2.2 Recurrent Networks as Directed Graphical Models

* As with a feedforward network, we usually wish to interpret the output of the RNN as a probability distribution, and we usually use the cross-entropy associated with that distribution to define the loss. 
* Mean squared error is the cross-entropy loss associated with an output distribution that is a unit Gaussian, for example, just as with a feedforward network.

<img src="figures/cap10.8.png" width=600 />
<img src="figures/cap10.9.png" width=600 />
<img src="figures/cap10.11.png" width=600 />

When we use a predictive log-likelihood training objective, such as Eq. 10.7, we train the RNN to estimate the conditional distribution of the next sequence element y(t) given the past inputs. This may mean that we maximize the log-likelihood

<img src="figures/cap10.20.png" width=600 />

or, if the model includes connections from the output at one time step to the next
time step,

<img src="figures/cap10.21.png" width=600 />

* Decomposing the joint probability over the sequence of y values as a series of one-step probabilistic predictions is one way to capture the full joint distribution across the whole sequence. 
* When we do not feed past y values as inputs that condition the next step prediction, the directed graphical model contains no edges from any y(i) in the past to the current y(t). In this case, the outputs y are conditionally independent given the sequence of x values. 
* When we do feed the actual y values (not their prediction, but the actual observed or generated values) back into the network, the directed graphical model contains edges from all y(i) values in the past to the current y (t) value.

#### As a simple example, 

let us consider the case where the RNN models only a sequence of scalar random variables Y = { y(1), . . . , y(τ) } , with no additional inputs x. The input at time step t is simply the output at time step t − 1. The RNN then defines a directed graphical model over the y variables.

We parametrize the joint distribution of these observations using the chain rule (Eq. 3.6) for conditional probabilities:

<img src="figures/cap10.23.png" width=600 />

where the right-hand side of the bar is empty for t = 1, of course.

Hence the negative log-likelihood of a set of values { y(1), . . . , y(τ) } according to such a model is

<img src="figures/cap10.24.png" width=600 />

* Many graphical models aim to achieve statistical and computational efficiency by omitting edges that do not correspond to strong interactions.
* For example, it is common to make the Markov assumption that the graphical model should only contain edges from {y(t−k) , . . . , y(t−1)} to y(t), rather than containing edges from the entire past history.

#### all past inputs

* However, in some cases, we believe that all past inputs should have an influence on the next element of the sequence. 
* RNNs are useful when we believe that the distribution over y(t) may depend on a value of y(i) from the distant past in a way that is not captured by the effect of y(i) on y(t−1)

#### ignoring the hidden units
* The graphical model over the y values with the complete graph structure is shown in Fig. 10.7. 
* The complete graph interpretation of the RNN is based on ignoring the hidden units h(t) by marginalizing them out of the model.

<img src="figures/cap10.22.png" width=600 />

#### including the hidden units

* The hidden units are a deterministic function of their inputs, so it may seem unusual to regard them as random variables. 
* However, it is perfectly legitimate to regard them as random variables. 
* The RNN defines conditional probability distribution over the hidden units given their inputs; the conditional distribution just happens to be a probability distribution that assigns probability 1 to a single state and probability 0 to all other states. 
* Including the hidden units in the graphical model reveals that the RNN provides a very efficient parametrization of the joint distribution over the observations.

<img src="figures/cap10.25.png" width=600 />

#### some computationally challenging

* Even with the efficient parametrization of the graphical model, some operations remain computationally challenging. 
* For example, it is difficult to predicting missing values in the middle of the sequence.
* The statistical complexity of the RNN may be flexibly adjusted by changing the complexity of the function f(h(t−1);x(t)) that produces the state variable.

#### The graphical model of recurrent networks

* The graphical model of recurrent networks is directed and decomposes the probability distribution as a product of conditionals without explicitly cutting any arc in the graphical model.
* The price recurrent networks pay for their reduced number of parameters is that <font color="red">optimizing the parameters</font> may be difficult.
* The <font color="red">parameter sharing</font> used in recurrent networks relies on the assumption that the same parameters can be used for different time steps.

To complete our view of an RNN as a graphical model, we must describe how to draw samples from the model.
* The main operation that we need to perform is simply to sample from the conditional distribution at each time step. 
* However, there is one additional complication. 
* The RNN must have some mechanism for determining the length of the sequence. This can be achieved in various ways.

<img src="figures/cap10.26.png" width=600 />

## 10.2.3 Modeling Sequences Conditioned on Context with RNNs

<img src="figures/cap10.8.png" width=600 />
<img src="figures/cap10.9.png" width=600 />

<img src="figures/cap10.20.png" width=600 />
<img src="figures/cap10.21.png" width=600 />

<img src="figures/cap10.22.png" width=600 />

<img src="figures/cap10.25.png" width=600 />

* In the previous section we described how an RNN could correspond to a directed graphical model over a sequence of random variables y(t) with no inputs x.
* In general, RNNs allow the extension of the graphical model view to represent not only a joint distribution over the y variables but also a conditional distribution over y given x.
* As discussed in the context of feedforward networks in Sec. 6.2.1.1, any model representing a variable P (y;θ) can be reinter- preted as model representing a conditional distribution P(y|ω ) with ω= θ.
* We can extend such a model to represent a distribution P(y | x) by using the same P(y | ω) as before, but making ω a function of x. 

#### fixed-size inputs
* Previously, we have discussed RNNs that take a sequence of vectors x(t) for t = 1,...,τ as input.
* Another option is to take only a single vector x as input. When x is a fixed-size vector, we can simply make it an extra input of the RNN that generates the y sequence.
* Some common ways of providing an extra input to an RNN are:
    1. as an extra input at each time step, or 
    2. as the initial state h(0), or
    3. both.

The first and most common approach is illustrated in Fig. 10.9.

<img src="figures/cap10.28.png" width=600 />

### a sequence of vectors x(t)

Rather than receiving only a single vector x as input, the RNN may receive a sequence of vectors x(t) as input.

#### conditional independence assumption

<img src="figures/cap10.27.png" width=600 />

#### to remove conditional independence assumption

* To remove the conditional independence assumption, we can add connections from the output at time t to the hidden unit at time t+ 1, as shown in Fig. 10.10.
* This kind of model representing a distribution over a sequence given another sequence still has one restriction, which is that the length of both sequences must be the same. We describe how to remove this restriction in Sec. 10.4.

<img src="figures/cap10.29.png" width=600 />

# 10.3 Bidirectional RNNs

* All of the recurrent networks we have considered up to now have a “causal” structure, meaning that the state at time t only captures information from the past, x(1), . . . , x(t−1), and the present input x(t) .
* Some of the models we have discussed also allow information from past y values to affect the current state when the y values are available.
* However, in many applications we want to output a prediction of y(t) which may depend on the whole input sequence.
     - For example, in speech recognition, the correct interpretation of the current sound as a phoneme may depend on the next few phonemes because of co-articulation and potentially may even depend on the next few words because of the linguistic dependencies between nearby words

* Bidirectional recurrent neural networks (or bidirectional RNNs) were invented
to address that need (Schuster and Paliwal, 1997). They have been extremely suc- cessful (Graves, 2012) in applications where that need arises, such as handwriting recognition (Graves et al., 2008; Graves and Schmidhuber, 2009), speech recogni- tion (Graves and Schmidhuber, 2005; Graves et al., 2013) and bioinformatics (Baldi
et al., 1999).
* As the name suggests, bidirectional RNNs combine an RNN that moves forward
through time beginning from the start of the sequence with another RNN that moves backward through time beginning from the end of the sequence.

<img src="figures/cap10.30.png" width=600 />

* This idea can be naturally extended to 2-dimensional input, such as images,by having four RNNs, each one going in one of the four directions: up, down, left, right.
* Compared to a convolutional network, RNNs applied to images are typically more expensive but allow for long-range lateral interactions between features in the same feature map

# 10.4 Encoder-Decoder Sequence-to-Sequence Architectures

* Here we discuss how an RNN can be trained to map an input sequence to an output sequence which is not necessarily of the same length.
* This comes up in many applications, such as speech recognition, machine translation or question answering, where the input and output sequences in the training set are generally not of the same length (although their lengths might be related).

#### context
* We often call the input to the RNN the “context.” We want to produce a representation of this context, C. The context C might be a vector or sequence of vectors that summarize the input sequence X = (x(1), . . . , x(nx ) ).

#### the encoder-decoder or sequence-to-sequence architecture
* The simplest RNN architecture for mapping a variable-length sequence to another variable-length sequence was first proposed by Cho et al. (2014a) and shortly after by Sutskever et al. (2014b).
* These authors respectively called this architecture, illustrated in Fig. 10.12, the encoder-decoder or sequence-to-sequence architecture.
* The idea is very simple: 
    - (1) an encoder or reader or input RNN processes the input sequence. 
        - The encoder emits the context C , usually as a simple function of its final hidden state. 
    - (2) a decoder or writer or output RNN is conditioned on that fixed-length vector (just like in Fig. 10.9) to generate the output sequence Y = (y(1) , . . . , y(ny) ). 
* In a sequence-to-sequence architecture, the two RNNs are trained jointly to maximize the average of logP(y(1),...,y(ny) |x(1),...,x(nx)) over all the pairs of x and y sequences in the training set. 
* The last state h_nx of the encoder RNN is typically used as a representation C of the input sequence that is provided as input to the decoder RNN.

<img src="figures/cap10.31.png" width=600 />

* If the context C is a vector, then the decoder RNN is simply a vector-to- sequence RNN as described in Sec. 10.2.3.
* One clear limitation of this architecture is when the context C output by the encoder RNN has a dimension that is too small to properly summarize a long sequence. 
    - This phenomenon was observed by Bahdanau et al. (2014) in the context of machine translation. 
    - They proposed to make C a variable-length sequence rather than a fixed-size vector. 
    - Additionally, they introduced an attention mechanism that learns to associate elements of the sequence C to elements of the output sequence.

# 10.5 Deep Recurrent Networks

* The computation in most RNNs can be decomposed into three blocks of parameters and associated transformations:
    1. from the input to the hidden state,
    2. from the previous hidden state to the next hidden state, and 
    3. from the hidden state to the output.

* With the RNN architecture of Fig. 10.3, each of these three blocks is associated with a single weight matrix. In other words, when the network is unfolded, each of these corresponds to a shallow transformation.

#### Would it be advantageous to introduce depth in each of these operations?

Experimental evidence (Graves et al., 2013; Pascanu et al., 2014a) strongly suggests so.

#### deep RNNs

* Graves et al. (2013) were the first to show a significant benefit of decomposing the state of an RNN into multiple layers as in Fig. 10.13 (left).
    - We can think of the lower layers in the hierarchy depicted in Fig. 10.13a as playing a role in transforming the raw input into a representation that is more appropriate, at the higher levels of the hidden state.
* Pascanu et al. (2014a) go a step further and propose to have a separate MLP (possibly deep) for each of the three blocks enumerated above, as illustrated in Fig. 10.13b.
    - Considerations of representational capacity suggest to allocate enough capacity in each of these three steps, but doing so by adding depth may hurt learning by making optimization difficult.
    - In general, it is easier to optimize shallower architectures, and adding the extra depth of Fig. 10.13b makes the shortest path from a variable in time step t to a variable in time step t + 1 become longer.
* However, as argued by Pascanu et al. (2014a), this can be mitigated by introducing skip connections in the hidden-to-hidden path, as illustrated in Fig. 10.13c.

<img src="figures/cap10.32.png" width=600 />

# 10.6 Recursive Neural Networks

#### 참고
* [6] CS224d: Deep Learning for Natural Language Processing : Recursive neural networks -- for parsing - http://cs224d.stanford.edu/lectures/CS224d-Lecture9.pdf

<img src="figures/cap10.33.png" width=600 />

* Recursive neural networks1 represent yet another generalization of recurrent networks, with a different kind of computational graph, which is <font color="red">structured as a deep tree, rather than the chain-like structure of RNNs</font>.
* Recursive networks have been successfully applied to processing data structures as input to neural nets (Frasconi et al., 1997, 1998), in natural language processing (Socher et al., 2011a,c, 2013a) as well as in computer vision (Socher et al., 2011b).
* <font color="red">One clear advantage of recursive nets</font> over recurrent nets is that for a sequence of the same length τ, the depth (measured as the number of compositions of nonlinear operations) can be drastically reduced from τ to O(logτ), which <font color="red">might help deal with long-term dependencies</font>. 
* <font color="red">An open question is how to best structure the tree</font>.
    - One option is to have a tree structure which does not depend on the data, such as a balanced binary tree.
        - For example, when processing natural language sentences, the <font color="red">tree structure for the recursive network can be fixed to the structure of the parse tree of the sentence</font> provided by a natural language parser (Socher et al., 2011a, 2013a).

# 10.7 The Challenge of Long-Term Dependencies

#### 참고
* [2] CS224d: Deep Learning for Natural Language Processing : Recurrent neural networks -- for language modeling and other tasks - http://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf
* [7] RNN Tutorial Part 3 - BPTT와 Vanishing Gradient 문제 - http://aikorea.org/blog/rnn-tutorial-3/
* [8] Gradient, Jacobian 행렬, Hessian 행렬, Laplacian - http://darkpgmr.tistory.com/132

The mathematical challenge of learning long-term dependencies in recurrent net- works was introduced in Sec. 8.2.5. 
* The basic problem is that gradients propagated over many stages tend to either <font color="red">vanish</font> (most of the time) or <font color="red">explode</font> (rarely, but with much damage to the optimization).
* Even if we assume that the parameters are such that the recurrent network is stable (can store memories, with gradients not exploding), the difficulty with long-term dependencies arises from the exponentially smaller weights given to long-term interactions (involving the multiplication of many Jacobians) compared to short-term ones.

<img src="http://www.wildml.com/wp-content/uploads/2015/10/rnn-bptt-with-gradients.png" width=600 />

<img src="http://nn.readthedocs.org/en/rtd/image/tanh.png" width=600 />

<img src="http://image.slidesharecdn.com/talk-140611223619-phpapp02/95/talk-18-638.jpg?cb=1402526450" width=600 />

<img src="http://nbviewer.jupyter.org/github/songorithm/ML/blob/master/part2/study05/dml08/figures/cap11.9.png" width=600 />

<img src="http://image.slidesharecdn.com/alecradfordodscpresentation-150603165045-lva1-app6891/95/recurrent-neural-networks-for-text-analysis-27-638.jpg?cb=1433350597" width=600 />

<img src="http://nbviewer.jupyter.org/github/songorithm/ML/blob/master/part2/study05/dml08/figures/cap11.10.png"  width=600 />

Recurrent networks involve the <font color="red">composition of the same function multiple times, once per time step</font>. These <font color="red">compositions can result in extremely nonlinear behavior</font>, as illustrated in Fig. 10.15.

<img src="figures/cap10.49_ppt.png" width=600 />

<img src="figures/cap10.34.png" width=600 />

<img src="figures/cap10.35.png" width=600 />

<font color="red">The eigenvalues are raised to the power of t causing eigenvalues with magnitude less than one to decay to zero and eigenvalues with magnitude greater than one to explode.</font>

<font color="blue">This problem is particular to recurrent networks</font>. 
* In the scalar case, imagine multiplying a weight w by itself many times. The product wt will either vanish or explode depending on the magnitude of w^(t). 
* However, if we make a non-recurrent  network that has a different weight w^(t) at each time step, the situation is different.

Specifically, whenever the model is able to <font color="red">represent long term dependencies</font>, the gradient of a long term interaction has exponentially smaller magnitude than the gradient of a short term interaction. 
* <font color="red">It does not mean that it is impossible to learn, but that it might take a very long time to learn long-term dependencies, because the signal about these dependencies will tend to be hidden by the smallest fluctuations arising from short-term dependencies.</font>
* In practice, the experiments in Bengio et al. (1994) show that as we increase the span of the dependencies that need to be captured, gradient-based optimization becomes increasingly difficult, with the probability of successful training of a traditional RNN via SGD rapidly reaching 0 for sequences of only length 10 or 20.

# 10.8 Echo State Networks

#### 참고
* [9] (Coursera, by Geoffrey Hinton) Neural Networks for Machine Learning : Lecture 8d Echo state networks - https://docs.google.com/viewer?url=https%3A%2F%2Fd396qusza40orc.cloudfront.net%2Fneuralnets%2Flecture_slides%2Flec8.pptx
* [10] Echo State Networks for Signal Processing - http://www.igi.tugraz.at/lehre/SeminarD/SS07/holzmann.pdf

Since learning the recurrent and input weights is difficult, one option that has been proposed (Jaeger, 2003; Maass et al., 2002; Jaeger and Haas, 2004; Jaeger, 2007b) is <font color="blue">to set the recurrent weights such that the recurrent hidden units do a good job of capturing the history of past inputs</font>, and <font color="red">only learn the output weights</font>. 
* This is the idea that was independently proposed for <font color="red">echo state networks or ESNs</font> (Jaeger and Haas, 2004; Jaeger, 2007b) and <font color="red">liquid state machines</font> (Maass et al., 2002).
* The latter is similar, except that it uses <font color="red">spiking neurons</font> (with binary outputs) instead of the continuous-valued hidden units used for ESNs. 
* Both ESNs and liquid state machines are termed <font color="red">reservoir computing</font> (Lukoševičius and Jaeger, 2009) to denote the fact that the <font color="blue">hidden units form of reservoir of temporal features which may capture different aspects of the history of inputs</font>

<img src="https://qph.is.quoracdn.net/main-qimg-9033824f1bc19bf1bf50ad0ab424ea94?convert_to_webp=true" width=600 />

#### kernel machines

One way to think about these reservoir computing recurrent networks is that they are similar to kernel machines: 
* they map an arbitrary length sequence (the history of inputs up to time t) into a fixed-length vector (the recurrent state h(t) ), on which a linear predictor (typically a linear regression) can be applied to solve the problem of interest.

#### The important question is therefore:

How do we set the input and recurrent weights so that a rich set of histories can be represented in the recurrent neural network state? 
* The answer proposed in the reservoir computing literature is to make the dynamical system associated with the recurrent net nearly be on the edge of stability.

#### spectral radius

#### 참고
* [11] Spectral Radius - http://mathworld.wolfram.com/SpectralRadius.html

The original idea was to make eigenvalues of the Jacobian of the state-to-state transition function be close to 1. As explained in Sec. 8.2.5, an important characteristic of a recurrent network is the eigenvalue spectrum of the Jacobians

J^(t) = ∂s^(t)/∂s^(t-1) Of particular importance is the spectral radius of J^(t) , 

defined to be the maximum of the absolute values of its eigenvalues.

<img src="figures/cap10.47_text.png" width=600 />

Of course, this example assumed that the Jacobian was the same at every time step, corresponding to a recurrent network with no nonlinearity. When a nonlinearity is present, the derivative of the nonlinearity will approach zero on many time steps, and help to prevent the explosion resulting from a large spectral radius. Indeed, the most recent work on echo state networks advocates <font color="red">using a spectral radius much larger than unity</font> 

The strategy of echo state networks is simply to fix the weights to have some spectral radius such as 3, where information is carried forward through time but does not explode due to the stabilizing effect of saturating nonlinearities like tanh

More recently, it has been shown that the techniques used to set the weights in ESNs could be used to initialize the weights in a fully trainable recurrent net-
work (with the hidden-to-hidden recurrent weights trained using back-propagation
through time), helping to learn long-term dependencies (Sutskever, 2012; Sutskever et al., 2013). In this setting, an initial spectral radius of 1.2 performs well, combined with the sparse initialization scheme described in Sec. 8.4.

#### 참고
* [12] Explorations in Echo State Networks -  http://www.ai.rug.nl/~mwiering/Thesis_Adrian_Millea.pdf

"The spectral radius is a critical tuning parameter for the echo state network. Usually the spectral radius is related to the input signal, in the sense that if lower timescale dynamics is expected (fast-oscillating signal) then a lower spectral-radius might be sufficient. However if longer memory is necessary then a higher spectral radius will be required. The downside of a bigger spectral radius is bigger time for the settling down of the network oscillations. Translating this into an experimental outcome means having a smaller region of optimality when searching for a good echo state network with respect to some dataset. The spectral radius is considered to be the most important tuning parameter by the creator of the ESN. Having a spectral radius bigger than 1, does not mean that the echo state network thus constructed will be necessary bad, but it gives very inconsistent results, thus making the search for a good ESN a much more random process than it already is."

# 10.9 Skip Connections through Time

<img src="http://deeplearning4j.org/img/lstm_skip_connection.png" />

<img src="figures/cap10.50_ppt.png" width=600 />

One way to deal with long-term dependencies is to <font color="red">add direct connections from variables in the distant path to variables in the present</font>. 
* The idea of using such skip connections dates back to Lin et al. (1996) and follows from the idea of incorporating delays in feedforward neural networks (Lang and Hinton, 1988). 
* In an ordinary recurrent network, a recurrent connection goes from a unit at time t to a unit at time t + 1. It is possible to construct recurrent networks with longer delays (Bengio, 1991).

As we have seen in Sec. 8.2.5, gradients may vanish or explode exponentially with respect to the number of time steps. Lin et al. (1996) introduced recurrent connections with a <font color="red">time-delay of d</font> to mitigate this problem.
* Gradients now diminish exponentially as a function of tau/d rather than τ . 
    - Since there are d both delayed and single step connections, gradients may still explode exponentially in τ . 
* This allows the learning algorithm to capture longer dependencies although not all long-term dependencies may be represented well in this way.

# 10.10 Leaky Units and a Spectrum of Diﬀerent Time Scales

#### 참고
* [13] Recurrent Neural Network - https://piazza.com/class_profile/get_resource/i48o74a0lqu0/i6ys94c8na8i2

<img src="figures/cap10.51_ppt.png" width=600 />

Another way to obtain paths on which the product of derivatives is close to one is to have units with linear self-connections and a weight near one on these connections.

#### leaky units

<img src="figures/cap10.48_text.png" width=600 />

* Skip connections through d time steps are a way of ensuring that a unit can always learn to be influenced by a value from d time steps earlier. 
* The use of a linear self-connection with a weight near one is a different way of ensuring that the unit can access values from the past. 
* The linear self-connection approach allows this effect to be adapted more smoothly and flexibly <font color="red">by adjusting the real-valued α rather than by adjusting the integer-valued skip length</font>.

There are two basic strategies for setting the time constants used by leaky units. 
* One strategy is to <font color="red">manually fix</font> them to values that remain constant, for example by sampling their values from some distribution once at initialization time. 
* Another strategy is to <font color="blue">make the time constants free parameters and learn them</font>. 
    - Having such leaky units at different time scales appears to help with long-term dependencies (Mozer, 1992; Pascanu et al., 2013a).

# 10.11 The Long Short-Term Memory and Other Gated RNNs
   * 10.11.1 LSTM
   * 10.11.2 Other Gated RNNs

As of this writing, the most effective sequence models used in practical applications are called <font color="red">gated RNNs</font>. 
* These include the <font color="blue">long short-term memory</font> and networks <font color="blue">based on the gated recurrent unit</font>.
* accumulate information
* forget the old state

## 10.11.1 LSTM

#### 참고
* [14] 엘에스티엠 네트워크 이해하기 - http://roboticist.tistory.com/m/post/571
* [15] RNN Tutorial Part 4 - GRU/LSTM RNN 구조를 Python과 Theano를 이용하여 구현하기 - http://aikorea.org/blog/rnn-tutorial-4/

<img src="http://artint.info/figures/ch07/sigmoidc.gif" width=600 />

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-chain.png" width=600 />

The clever idea of introducing <font color="red">self-loops to produce paths where the gradient can flow for long durations</font> is a core contribution of the initial <font color="blue">long short-term memory (LSTM) model</font> (Hochreiter and Schmidhuber, 1997).
* A crucial addition has been to <font color="red">make the weight</font> <font color="blue">on this self-loop</font> <font color="red">conditioned on the context</font>, rather than fixed (Gers et al., 2000).

The LSTM has been found extremely successful in many applications, such as 
* unconstrained handwriting recognition (Graves et al., 2009), 
* speech recognition (Graves et al., 2013; Graves and Jaitly, 2014), 
* handwriting generation (Graves, 2013), 
* machine translation (Sutskever et al., 2014a), 
* image captioning (Kiros et al., 2014b; Vinyals et al., 2014b; Xu et al., 2015b) and 
* parsing (Vinyals et al., 2014a).

<font color="red">Instead of a unit that simply applies an element-wise nonlinearity</font> to the affine transformation of inputs and recurrent units, LSTM
recurrent networks have <font color="blue">“LSTM cells” that have an internal recurrence (a self-loop)</font>,
in addition to the outer recurrence of the RNN.

<img src="figures/cap10.36.png" width=600 />

Each cell has the same inputs and outputs as an ordinary recurrent network, but has more parameters and a system of gating units that controls the flow of information.

#### state unit

The most important component is the state unit s(t) that has a linear self-loop similar to the leaky i
units described in the previous section.

#### forget gate unit & external input gate unit

However, here, the self-loop weight (or the associated time constant) is controlled by a forget gate unit fi(t) (for time step t and cell i), that sets this weight to a value between 0 and 1 via a sigmoid unit:

<img src="figures/cap10.37.png" width=600 />

#### output gate

<img src="figures/cap10.38.png" width=600 />

<font color="red">LSTM networks have been shown to learn long-term dependencies more easily than the simple recurrent architectures</font>, first on artificial data sets designed for testing the ability to learn long-term dependencies (Bengio et al., 1994; Hochreiter and Schmidhuber, 1997; Hochreiter et al., 2000), then on challenging sequence processing tasks where state-of-the-art performance was obtained (Graves, 2012; Graves et al., 2013; Sutskever et al., 2014a).

## 10.11.2 Other Gated RNNs

Which pieces of the LSTM architecture are actually necessary? What other successful architectures could be designed that allow the network to dynamically control the time scale and forgetting behavior of different units?

### GRUs

Some answers to these questions are given with the recent work on gated RNNs, whose units are also known as gated recurrent units or GRUs (Cho et al., 2014b; Chung et al., 2014, 2015a; Jozefowicz et al., 2015b; Chrupala et al., 2015)

#### 참고
* [14] 엘에스티엠 네트워크 이해하기 - http://roboticist.tistory.com/m/post/571

<img src="http://colah.github.io/posts/2015-08-Understanding-LSTMs/img/LSTM3-var-GRU.png" width=600 />

The main difference with the LSTM is that a <font color="red">single gating unit simultaneously controls</font> <font color="blue">the forgetting factor and the decision to update the state unit</font>.

#### update gate & reset gate

The update equations are the following:

<img src="figures/cap10.40.png" width=600 />

<img src="figures/cap10.41.png" width=600 />

* The reset and updates gates can individually “ignore” parts of the state vector. 
* The update gates act like conditional leaky integrators that can linearly gate any dimension, thus 
    - choosing to copy it (at one extreme of the sigmoid) or 
    - completely ignore it (at the other extreme) by replacing it by the new “target state” value (towards which the leaky integrator wants to converge). 
* The reset gates control which parts of the state get used to compute the next target state, introducing an additional nonlinear effect in the relationship between past state and future state

# 10.12 Optimization for Long-Term Dependencies
   * 10.12.1 Clipping Gradients

## 10.12.1 Clipping Gradients

<img src="figures/cap10.43.png" width=600 />

<img src="figures/cap10.44.png" width=600 />

# 10.13 Regularizing to Encourage Information Flow

<img src="figures/cap10.45.png" width=600 />
<img src="figures/cap10.46.png" width=600 />

# 10.14 Organizing the State at Multiple Time Scales

# 10.15 Explicit Memory

<img src="figures/cap10.42.png" width=600 />

# 참고자료
* [1] Bengio's deep learning book / Chapter 10. Sequence Modeling: Recurrentand Recursive Nets - http://www.deeplearningbook.org/contents/rnn.html
* [2] CS224d: Deep Learning for Natural Language Processing : Recurrent neural networks -- for language modeling and other tasks - http://cs224d.stanford.edu/lectures/CS224d-Lecture7.pdf
* [3] CS231n: Convolutional Neural Networks for Visual Recognition (2015) - Beyond Image Classification: localization, detection, segmentation - http://vision.stanford.edu/teaching/cs231n/slides/lecture11.pdf 
Recurrent Networks I: Image Captioning example
* [4] CS231n: Convolutional Neural Networks for Visual Recognition (2016) - Recurrent Neural Networks (RNN), Long Short Term Memory (LSTM) - http://cs231n.stanford.edu/slides/winter1516_lecture10.pdf
 * [5] Probabilistic Graphical Models : Template Models - http://spark-university.s3.amazonaws.com/stanford-pgm/slides/Section-2-Representation-Template-Models.pdf
* [6] CS224d: Deep Learning for Natural Language Processing : Recursive neural networks -- for parsing - http://cs224d.stanford.edu/lectures/CS224d-Lecture9.pdf
* [7] RNN Tutorial Part 3 - BPTT와 Vanishing Gradient 문제 - http://aikorea.org/blog/rnn-tutorial-3/
* [8] Gradient, Jacobian 행렬, Hessian 행렬, Laplacian - http://darkpgmr.tistory.com/132
* [9] (Coursera, by Geoffrey Hinton) Neural Networks for Machine Learning : Lecture 8d Echo state networks - https://docs.google.com/viewer?url=https%3A%2F%2Fd396qusza40orc.cloudfront.net%2Fneuralnets%2Flecture_slides%2Flec8.pptx
* [10] Echo State Networks for Signal Processing - http://www.igi.tugraz.at/lehre/SeminarD/SS07/holzmann.pdf
* [11] Spectral Radius - http://mathworld.wolfram.com/SpectralRadius.html
* [12] Explorations in Echo State Networks - http://www.ai.rug.nl/~mwiering/Thesis_Adrian_Millea.pdf
* [13] Recurrent Neural Network - https://piazza.com/class_profile/get_resource/i48o74a0lqu0/i6ys94c8na8i2
* [14] 엘에스티엠 네트워크 이해하기 - http://roboticist.tistory.com/m/post/571
* [15] RNN Tutorial Part 4 - GRU/LSTM RNN 구조를 Python과 Theano를 이용하여 구현하기 - http://aikorea.org/blog/rnn-tutorial-4/